In [1]:
!pip install git+https://github.com/huggingface/diffusers.git@1870fb05a903546b79236d277ae4bc12e626b328 transformers accelerate imageio[ffmpeg]

  Cloning https://github.com/huggingface/diffusers.git (to revision 1870fb05a903546b79236d277ae4bc12e626b328) to c:\users\chris\appdata\local\temp\pip-req-build-prao4fqe
  Resolved https://github.com/huggingface/diffusers.git to commit 1870fb05a903546b79236d277ae4bc12e626b328
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/13/30/54b59e73400df3de506ad8630284e9fd63f4b94f735423d55fc342181037/transformers-4.33.1-py3-none-any.whl.metadata
  Using cached transformers-4.33.1-py3-none-any.whl.metadata (119 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/4d/a7/05c67003d659

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git 'C:\Users\chris\AppData\Local\Temp\pip-req-build-prao4fqe'
  Running command git rev-parse -q --verify 'sha^1870fb05a903546b79236d277ae4bc12e626b328'
  Running command git fetch -q https://github.com/huggingface/diffusers.git 1870fb05a903546b79236d277ae4bc12e626b328
  Running command git checkout -q 1870fb05a903546b79236d277ae4bc12e626b328


In [2]:
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux
!pip install -q -U xformers
!pip install -q matplotlib

In [3]:
import diffusers
import transformers
print(diffusers.__version__, transformers.__version__)
# try:
#     import xformers
#     print(xformers.__version__)
# except:
#     print("xformers not installed correctly")

c:\Users\chris\anaconda3\envs\txt2vid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.15.0.dev0 4.33.1


In [4]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler, StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import export_to_video
import cv2
from PIL import Image
import numpy as np

import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def dummy(images, **kwargs):
    return images, False

pipe = DiffusionPipeline.from_pretrained("./diffusion_pipeline/", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.safety_checker = dummy

controlnet = [
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16),
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16),
]
controlpipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
controlpipe.scheduler = UniPCMultistepScheduler.from_config(controlpipe.scheduler.config)
controlpipe.enable_model_cpu_offload()
controlpipe.safety_checker = dummy
#controlpipe.enable_xformers_memory_efficient_attention()

text_encoder\model.safetensors not found
Fetching 15 files:   7%|▋         | 1/15 [00:00<00:09,  1.49it/s]































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [5]:
# Save these if you never want to have to download them again.
#pipe.save_pretrained("./diffusion_pipeline/")
#controlpipe.save_pretrained("./controlnet_pipeline/")

In [6]:
prompt = 'Darth Maul breakdancing in the forest, best quality, masterpiece, photorealistic, unreal engine 5'
negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality, bad face,"
video_duration_seconds = 3
num_frames = video_duration_seconds * 10
video_frames = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=25, num_frames=num_frames).frames
video_path = export_to_video(video_frames)

100%|██████████| 25/25 [00:13<00:00,  1.84it/s]


In [7]:
def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video = imageio.mimread(video_path)  #Loading video
writer = imageio.get_writer('./videov1.mp4', fps=10)
for frame in video:
    writer.append_data(frame)
writer.close()

In [8]:
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image

for i in range(len(video_frames)):
    image = np.array(video_frames[i])

    low_threshold = 100
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    canny_image = Image.fromarray(image)
    #canny_image.show()
    openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
    openpose_image = openpose(video_frames[i])
    
    images = [openpose_image, canny_image]
    output = controlpipe(
        'Darth Maul breakdancing in a desert, best quality, masterpiece, photorealistic, unreal engine 5',
        images,
        negative_prompt=negative_prompt,
        generator=torch.Generator(device='cpu'),
        num_inference_steps=20,
    )
    output.images[0].save(f'./temp_imgs/controlnet-{i}.png')

c:\Users\chris\anaconda3\envs\txt2vid\lib\site-packages\controlnet_aux\mediapipe_face\mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
100%|██████████| 20/20 [00:02<00:00,  7.64it/s]


In [9]:
from pathlib import Path

writer = imageio.get_writer('./videov2.mp4', fps=10)

for file in Path("./temp_imgs/").iterdir():
    if not file.is_file():
        continue
    im = imageio.imread(file)
    writer.append_data(im)
writer.close()

C:\Users\chris\AppData\Local\Temp\ipykernel_4136\3214315076.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imageio.imread(file)


In [10]:
canny_image.show()
openpose_image.show()